<a href="https://colab.research.google.com/github/CodeMonkey01/DataMining1/blob/main/SVM_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from google.colab import files
#df = files.upload()


In [ ]:
import pandas as pd

df = pd.read_csv('IMDB Dataset.csv')
df.describe(include='all')

df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
%pip install tensorflow-text
import pandas as pd
from sklearn.preprocessing import LabelEncoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.9 MB 6.7 MB/s 
     |████████████████████████████████| 578.0 MB 11 kB/s 
     |████████████████████████████████| 5.9 MB 46.3 MB/s 
     |████████████████████████████████| 1.7 MB 54.5 MB/s 
     |████████████████████████████████| 438 kB 42.8 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.1

# **Getting an overview over the Data:**

In [ ]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [ ]:
df[df["sentiment"]== "positive"]

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
...,...,...
49983,"I loved it, having been a fan of the original ...",positive
49985,Imaginary Heroes is clearly the best film of t...,positive
49989,I got this one a few weeks ago and love it! It...,positive
49992,John Garfield plays a Marine who is blinded by...,positive


In [ ]:
df[df["sentiment"]== "negative"]

,review,sentiment
3,Basically there's a family where a little boy ...,negative
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
10,Phil the Alien is one of those quirky films wh...,negative
11,I saw this movie when I was about 12 when it c...,negative
...,...,...
49994,This is your typical junk comedy.<br /><br />T...,negative
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


##**Check for imbalance**

In [ ]:
df_pos = df[df["sentiment"]== "positive"]
df_neg = df[df["sentiment"]== "negative"]

#drop the difference between the positive and negative set.

df_raw_pos = df.iloc[186: , : ]
df_raw = pd.concat([df_raw_pos,df], ignore_index=True)
df_raw.sample(frac=1).reset_index(drop=True)

#Check for balance now.

print(df_raw["sentiment"].value_counts())

positive    49922
negative    49892
Name: sentiment, dtype: int64


In [ ]:
df_pos.describe()

,review,sentiment
count,25000,25000
unique,24884,1
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [ ]:
df_neg.describe()

,review,sentiment
count,25000,25000
unique,24698,1
top,This show comes up with interesting locations ...,negative
freq,3,25000


# **Preprocessing**

Example:

In [ ]:
import re
#Removes Punctuations
def remove_punctuations(data):
    punct_tag=re.compile(r'[^\w\s]')
    data=punct_tag.sub(r'',data)
    return data

#Removes HTML syntaxes
def remove_html(data):
    html_tag=re.compile(r'<.*?>')
    data=html_tag.sub(r'',data)
    return data

#Removes URL data
def remove_url(data):
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data

#Removes Emojis
def remove_emoji(data):
    emoji_clean= re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    data=emoji_clean.sub(r'',data)
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data

df['review']=df['review'].apply(lambda z: remove_punctuations(z)).apply(lambda z: remove_html(z))
df['review']=df['review'].apply(lambda z: remove_url(z)).apply(lambda z: remove_emoji(z))
# Remove all the special characters
df['review']=df['review'].apply(lambda z: re.sub(r'\W', ' ', z))
# remove all single characters
df['review']=df['review'].apply(lambda z: re.sub(r'\s+[a-zA-Z]\s+', ' ', z))
# Remove single characters from the start
df['review']=df['review'].apply(lambda z: re.sub(r'\^[a-zA-Z]\s+', ' ', z)) 
# Substituting multiple spaces with single space
df['review']=df['review'].apply(lambda z: re.sub(r'\s+', ' ', z, flags=re.I))
# Removing prefixed 'br'
df['review']=df['review'].apply(lambda z: re.sub(r'^br\s+', '', z))
# if you want to turn reviews into lists. use this. (algorithm dependent)
#df_data['review']=df_data['review'].apply(lambda z: z.split())
#encode target to numeric

# Transform class from Boolean to integer value.

df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x == "positive" else 0)

In [ ]:
#Stemming

import re
import nltk
from nltk.stem import PorterStemmer

token_pattern = re.compile(r"(?u)\b\w\w+\b")

ps = PorterStemmer()

nltk.download('punkt')
nltk.download('stopwords')

df['stemmed']=df['review'].apply(lambda x: ' '.join([ps.stem(y) for y in token_pattern.findall(x)]))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# **SVM**



**Define Target variable and redefine data for training**

In [ ]:
df_target = df['sentiment']
df_data = df.copy()
df_data = df_data.drop(columns='review')
df_data = df_data.rename({'sentiment': 'sentiment_variable'}, axis = 1)

df_data.head(5)

,sentiment_variable,stemmed
0,1,one of the other review ha mention that after ...
1,1,wonder littl product br br the film techniqu i...
2,1,thought thi wa wonder way to spend time on too...
3,0,basic there famili where littl boy jake think ...
4,1,petter mattei love in the time of money is vis...


Vectorize stemmed_stop_removed column

In [ ]:
# Tfidf vectorizer

from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
vectorizer = TfidfVectorizer(min_df=0.001)
matrix = vectorizer.fit_transform(df_data['stemmed'])
df_data = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names())


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#pd.set_option('max_columns', None)
df_data.head(5)

,010,10,100,1000,10000,101,1010,10br,11,110,...,yr,yup,zane,zani,zealand,zellweg,zero,zombi,zone,zoom
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.073089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.105471,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


Here we train the final model with the identified parameters and calculate the scores accuracy afterwards.

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create train/test split
df_data_train, df_data_test, df_target_train, df_target_test = train_test_split(
    df_data, df_target, test_size=0.2, random_state=42)

#SVM classifier with parameters
svm = LinearSVC(random_state=42, tol=0.1,C=1,dual=True,penalty='l2', loss='hinge',max_iter=300000)

#train final model
svm.fit(df_data_train, df_target_train)

#test final model
df_prediction = svm.predict(df_data_test)

print("Accuracy: {}".format(accuracy_score(df_target_test, df_prediction)))

Accuracy: 0.893


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

#also try out different evaluation measures to make sure there are no differences
print("Precision: {}".format(precision_score(df_target_test,df_prediction)))
print("Recall: {}".format(recall_score(df_target_test,df_prediction)))
print("F1: {}".format(f1_score(df_target_test,df_prediction)))

Precision: 0.8845185041658593
Recall: 0.9059337170073427
F1: 0.8950980392156862
